# Thinkful - 4.4.2 - Challenge - NLP Supervised

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.metrics import accuracy_score
from collections import Counter

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
#persuasion = gutenberg.raw('Data/austen-persuasion.txt')
#alice = gutenberg.raw('Data/carroll-alice.txt')
filename1 = 'Data/austen-persuasion.txt'
file1 = open(filename1,'rt')
persuasion = file1.read()
file1.close()

filename2 = 'Data/carroll-alice.txt'
file2 = open(filename2,'rt')
alice = file2.read()
file2.close()

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(ALICE, ’S, ADVENTURES, IN, WONDERLAND, Lewis,...",Carroll
1,"(THE, MILLENNIUM, FULCRUM, EDITION, 3.0, Alice...",Carroll
2,"(So, she, was, considering, in, her, own, mind...",Carroll
3,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
4,"(Oh, dear, !)",Carroll


**Parts of Speech Bags Generator**

In [3]:
# Utility function to create a list of the potential parts of speech
def parts_of_speech(text):
    
    # Filter out punctuation and stop words.
    parts_speech = [token.pos_ for token in text]
    
    # Return all words.
    #return [item[0] for item in Counter(parts_speech)]
    return parts_speech

# Set up the bags.
alicepos = parts_of_speech(alice_doc)
persuasionpos = parts_of_speech(persuasion_doc)

# Combine bags to create a set of unique words.
pos_words = list(set(alicepos + persuasionpos))

print(pos_words)

['ADV', 'PROPN', 'ADJ', 'ADP', 'X', 'DET', 'NUM', 'PUNCT', 'CCONJ', 'PRON', 'PART', 'NOUN', 'INTJ', 'VERB']


**Parts of Speech Counter**

In [4]:
# Creates a data frame with features for each part of speech corresponding to each word in our part of speech word set.
# Each value is the count of the times the part of speech appears in each sentence.
def pos_features(sentences, pos_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df1 = pd.DataFrame(columns=pos_words)
    df1['text_sentence'] = sentences[0]
    df1['text_source'] = sentences[1]
    df1.loc[:, pos_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df1['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        pos = [token.pos_ for token in sentence]
        df1['length'] = len(sentence)
        
        # Populate the row with word counts.
        for pos1 in pos:
            df1.loc[i, pos1] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
    
    return df1
        
# Create our data frame with features. This can take a while to run.
pos_counts = pos_features(sentences, pos_words)
pos_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,ADV,PROPN,ADJ,ADP,X,DET,NUM,PUNCT,CCONJ,PRON,PART,NOUN,INTJ,VERB,text_sentence,text_source,length
0,0,5,0,1,0,0,0,0,0,0,1,0,0,0,"(ALICE, ’S, ADVENTURES, IN, WONDERLAND, Lewis,...",Carroll,1
1,3,6,3,8,0,6,1,6,6,3,2,11,0,13,"(THE, MILLENNIUM, FULCRUM, EDITION, 3.0, Alice...",Carroll,1
2,7,2,7,8,0,6,0,7,2,4,1,8,0,11,"(So, she, was, considering, in, her, own, mind...",Carroll,1
3,6,2,1,4,0,3,0,3,1,2,1,2,0,5,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,1
4,0,0,0,0,0,0,0,1,0,0,0,0,2,0,"(Oh, dear, !)",Carroll,1


**Common Words Bags Generator**

In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

**Word Counter**

In [6]:
# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df2 = pd.DataFrame(columns=common_words)
    df2['text_sentence'] = sentences[0]
    df2['text_source'] = sentences[1]
    df2.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df2['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words)]
        
        # Populate the row with word counts.
        for word in words:
            df2.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
            
    return df2

# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,directly,morcar,drown,new,warm,heartily,ascertain,similar,clean,farm,...,setting,destroy,absurd,recollect,cousin,stoop,minute,inconvenient,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(ALICE, ’S, ADVENTURES, IN, WONDERLAND, Lewis,...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(THE, MILLENNIUM, FULCRUM, EDITION, 3.0, Alice...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll


In [7]:
pos_counts2 = pos_counts.drop(['text_sentence','text_source'], axis=1)
df = pd.concat([pos_counts2, word_counts], axis=1)
df.head()

,ADV,PROPN,ADJ,ADP,X,DET,NUM,PUNCT,CCONJ,PRON,...,setting,destroy,absurd,recollect,cousin,stoop,minute,inconvenient,text_sentence,text_source
0,0,5,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(ALICE, ’S, ADVENTURES, IN, WONDERLAND, Lewis,...",Carroll
1,3,6,3,8,0,6,1,6,6,3,...,0,0,0,0,0,0,0,0,"(THE, MILLENNIUM, FULCRUM, EDITION, 3.0, Alice...",Carroll
2,7,2,7,8,0,6,0,7,2,4,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
3,6,2,1,4,0,3,0,3,1,2,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll


# Models: Original

**Bag of Words with Random Forest Classifier**

In [8]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = word_counts.drop(['text_sentence','text_source'], 1)

X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.4,random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.987094743469

Test set score: 0.878716375649


**Bag of Words with Random Forest Classifier**

In [10]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = df['text_source']
X = np.array(df.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.4,random_state=0)
train = rfc.fit(X_train, y_train)

rfc_train_1 = rfc.score(X_train, y_train)
rfc_test_1 = rfc.score(X_test, y_test)

print('Training set score:', rfc_train_1)
print('\nTest set score:', rfc_test_1)

Training set score: 0.988039030532

Test set score: 0.882491741387


**Bag of Words with Logistic Regression**

In [11]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)

lr_train_1 = lr.score(X_train, y_train)
lr_test_1 = lr.score(X_test, y_test)

print('Training set score:', lr_train_1)
print('\nTest set score:', lr_test_1)

(3177, 3275) (3177,)
Training set score: 0.946175637394

Test set score: 0.893345917886


**Bag of Words with Gradient Boosting**

In [12]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

clf_train_1 = clf.score(X_train, y_train)
clf_test_1 = clf.score(X_test, y_test)

print('Training set score:', clf_train_1)
print('\nTest set score:', clf_test_1)

Training set score: 0.90462700661

Test set score: 0.886267107126


**Bag of Words with Support Vector Classifier**

In [13]:
from sklearn.svm import SVC

svm = SVC(kernel = 'linear')
svm.fit(X_train, y_train)

svm_train_1 = svm.score(X_train, y_train)
svm_test_1 = svm.score(X_test, y_test)

print('Training set score:', svm_train_1)
print('\nTest set score:', svm_test_1)

Training set score: 0.98363235757

Test set score: 0.907031618688


# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%. 
* See what you can do to improve performance. Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires. 
* Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.

**Bag of Words and Parts of Speech with Random Forest Classifier**

In [14]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = df['text_source']
X = np.array(df.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.4,random_state=0)
train = rfc.fit(X_train, y_train)

rfc_train_2 = rfc.score(X_train, y_train)
rfc_test_2 = rfc.score(X_test, y_test)

print('Training set score:', rfc_train_2)
print('\nTest set score:', rfc_test_2)

Training set score: 0.98929807995

Test set score: 0.875412930628


**Bag of Words and Parts of Speech with Logistic Regression**

In [15]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)

lr_train_2 = lr.score(X_train, y_train)
lr_test_2 = lr.score(X_test, y_test)

print('Training set score:', lr_train_2)
print('\nTest set score:', lr_test_2)

(3177, 3275) (3177,)
Training set score: 0.946175637394

Test set score: 0.893345917886


**Bag of Words and Parts of Speech with Gradient Boosting**

In [16]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

clf_train_2 = clf.score(X_train, y_train)
clf_test_2 = clf.score(X_test, y_test)

print('Training set score:', clf_train_2)
print('\nTest set score:', clf_test_2)

Training set score: 0.90462700661

Test set score: 0.885323265691


**Bag of Words and Parts of Speech with Support Vector Machines**

In [17]:
svm = SVC(kernel = 'linear')
svm.fit(X_train, y_train)

svm_train_2 = svm.score(X_train, y_train)
svm_test_2 = svm.score(X_test, y_test)

print('Training set score:', svm_train_2)
print('\nTest set score:', svm_test_2)

Training set score: 0.98363235757

Test set score: 0.907031618688


**Summary**

In [20]:
summary = np.array([['Random Forest Classifier',"%0.3f" % rfc_test_1,
                      "%0.3f" % rfc_test_2],
                  ['Logistic Regression',"%0.3f" % lr_test_1,
                      "%0.3f" % lr_test_2],
                  ['Gradient Boosting Regression',"%0.3f" % clf_test_1,
                      "%0.3f" % clf_test_2],
                  ['Support Vector Machines',"%0.3f" % svm_test_1,
                      "%0.3f" % svm_test_2]])

df_summary = pd.DataFrame(summary)
df_summary.columns = ['Model','BoW','BoW with PoS']

df_summary.head()

,Model,BoW,BoW with PoS
0,Random Forest Classifier,0.882,0.875
1,Logistic Regression,0.893,0.893
2,Gradient Boosting Regression,0.886,0.885
3,Support Vector Machines,0.907,0.907


# Challenge 1:

* Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work. This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

In [22]:
# Clean the Emma data.
filename3 = 'Data/austen-emma.txt'
file3 = open(filename3,'rt')
emma = file3.read()
file3.close()

emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

EMMA By Jane Austen Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy di


In [25]:
# Parse our cleaned data.
emma_doc = nlp(emma)

# Group into sentences.
emma_sents = [[sent, "Emma"] for sent in emma_doc.sents]

print("Alice # of sentences: ",len(alice_sents))
print("Emma # of sentences: ",len(emma_sents))

Alice # of sentences:  1645
Emma # of sentences:  12265


In [26]:
# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)

# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_bow = bow_features(emma_sentences, common_words)

# Create our data frame with features. This can take a while to run.
emma_pos_counts = pos_features(emma_sentences, pos_words)

emma_pos_counts = emma_pos_counts.drop(['text_sentence','text_source'], axis=1)
df1 = pd.concat([emma_pos_counts, emma_bow], axis=1)
df1.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 0
Processing row 500
Processing row 1000
Processing row 1500


,ADV,PROPN,ADJ,ADP,X,DET,NUM,PUNCT,CCONJ,PRON,...,setting,destroy,absurd,recollect,cousin,stoop,minute,inconvenient,text_sentence,text_source
0,2,5,7,6,0,4,2,8,4,1,...,0,0,0,0,0,0,0,0,"(EMMA, By, Jane, Austen, Emma, Woodhouse, ,, h...",Emma
1,2,0,6,6,0,4,1,5,1,1,...,0,0,0,0,0,0,0,0,"(She, was, the, youngest, of, the, two, daught...",Emma
2,3,0,8,7,0,3,0,3,1,1,...,0,0,0,0,0,0,0,0,"(Her, mother, had, died, too, long, ago, for, ...",Emma
3,2,5,2,5,0,3,1,4,1,0,...,0,0,0,0,0,0,0,0,"(Sixteen, years, had, Miss, Taylor, been, in, ...",Emma
4,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,"(Between, _, them)",Emma


In [32]:
# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((X_train[y_train[y_train=='Carroll'].index],
    df1.drop(['text_sentence','text_source'], 1)), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],pd.Series(['Austen'] * df1.shape[0])])

**Logistic Regression Model**

In [33]:
# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.682797731569


col_0,Austen,Carroll
row_0,,
Austen,1518,127
Carroll,712,288


**Support Vector Machines Model**

In [30]:
# Model.
print('\nTest set score:', svm.score(X_Emma_test, y_Emma_test))
svm_Emma_predicted = svm.predict(X_Emma_test)
pd.crosstab(y_Emma_test, svm_Emma_predicted)


Test set score: 0.526654064272


col_0,Austen,Carroll
row_0,,
Austen,1067,578
Carroll,674,326


**Gradient Boosting Classifier Model**

In [31]:
# Model.
print('\nTest set score:', clf.score(X_Emma_test, y_Emma_test))
clf_Emma_predicted = clf.predict(X_Emma_test)
pd.crosstab(y_Emma_test, clf_Emma_predicted)


Test set score: 0.522117202268


col_0,Austen,Carroll
row_0,,
Austen,1141,504
Carroll,760,240
